In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

tf.set_random_seed(4444)


In [48]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.33
sess = tf.InteractiveSession(config = config)

### Loading all the signals

In [4]:
import librosa
files = np.asarray(librosa.util.find_files('/N/u/priyadarshini/Projects/Noise_signals/tr'))


### Creating Magnitude Spectrograms for all Train signals

In [5]:
# Noise Signals
N = []
for noise_signal in files[0:1200]:
    sn, sr = librosa.load(noise_signal, sr=None)
    Sn = librosa.stft(sn, n_fft=1024, hop_length=512)
    S_noise = np.transpose(np.abs(Sn))
    N.append(S_noise)

# Clean Speech Signals    
S = []
for clean_signal in files[1200:2400]:
    ss, sr = librosa.load(clean_signal, sr=None)
    Ss = librosa.stft(ss, n_fft=1024, hop_length=512)
    S_clean = np.transpose(np.abs(Ss))
    S.append(S_clean)    


# Mixed Noisy Speech Signals    
X = []
for mix_signal in files[2400:3600]:
    sx, sr = librosa.load(mix_signal, sr=None)
    Sx = librosa.stft(sx, n_fft=1024, hop_length=512)
    S_mix = np.transpose(np.abs(Sx))
    X.append(S_mix)    

### Creating IBM Matrix per spectogram - Training signals

In [16]:
IBM = X
for k in range(0,len(IBM)) :
    for i in range(0,len(IBM[k])):
        for j in range(0,len(IBM[k][i])):
            if S[k][i][j] > N[k][i][j]:
                IBM[k][i][j] = 1
            else:
                IBM[k][i][j] = 0

In [17]:
max = 0
for spectogram in X :
    rows = len(spectogram)
    if rows >max:
        max = rows
print(max)        

178


### Padding train matrices with zeroes

In [18]:
X_m = [np.zeros((178,513))]*1200
for k in range(0,len(X)) :
    for i in range(0,len(X[k])):
        for j in range(0,len(X[k][i])):
            X_m[k][i][j] = X[k][i][j]

IBM_m = [np.zeros((178,513))]*1200
for k in range(0,len(IBM)) :
    for i in range(0,len(IBM[k])):
        for j in range(0,len(IBM[k][i])):
            IBM_m[k][i][j] = IBM[k][i][j]            
    

In [20]:
X_new = np.array(X_m).reshape(-1,5,513)  
IBM_new = np.array(IBM_m).reshape(-1,5,513)
print(X_new.shape)
print(IBM_new.shape)

(42720, 5, 513)
(42720, 5, 513)


### Parameters for the RNN network

In [49]:
num_inputs = 513

num_time_steps = 10

num_neurons = 128

num_outputs = 513

learning_rate = 0.001 

num_epochs = 20

batch_size = 64

In [50]:
x = tf.placeholder(tf.float32, [None, None, num_inputs])
y = tf.placeholder(tf.float32, [None, None, num_outputs])

### Defining RNN network 

In [51]:
cell = tf.contrib.rnn.OutputProjectionWrapper(
    tf.contrib.rnn.BasicLSTMCell(num_units=num_neurons, activation=tf.nn.sigmoid),
    output_size=num_outputs) 
cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob = 0.99)

outputs, states = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32);
loss = tf.reduce_mean(tf.square(outputs - y)) # MSE
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train = optimizer.minimize(loss)

### Batching function

In [56]:
def batching_1(X, IBM, batch_size):
        for start_index in range(0, X.shape[0] - batch_size + 1, batch_size):    
            index = slice(start_index, start_index + batch_size)
            yield X[index], IBM[index]

### Training RNN Network - Acheived train MSE loss of 0.0038 

In [57]:
sess.run(tf.global_variables_initializer())

for i in range(0,num_epochs):
     for batch in batching_1(X_new, IBM_new, batch_size):
        X_batch, y_batch = batch
        sess.run(train, feed_dict={x: X_batch, y: y_batch})
        
     if i % 1 == 0:
            
            acc = loss.eval(feed_dict={x: X_batch, y: y_batch})
            print(i, "\tLoss:", acc)

0 	Loss: 0.016333217
1 	Loss: 0.0067119976
2 	Loss: 0.004720248
3 	Loss: 0.0038714337
4 	Loss: 0.0038870147
5 	Loss: 0.004064526
6 	Loss: 0.0037911918
7 	Loss: 0.0040458255
8 	Loss: 0.0037615318
9 	Loss: 0.0037018957
10 	Loss: 0.003832265
11 	Loss: 0.003865591
12 	Loss: 0.0038263637
13 	Loss: 0.0038724015
14 	Loss: 0.0038652164
15 	Loss: 0.0039139353
16 	Loss: 0.0037297388
17 	Loss: 0.003745436
18 	Loss: 0.003848712
19 	Loss: 0.003852201


### Validation set - Creating Magnitude Spectrograms for all signals

In [7]:
files_v = np.asarray(librosa.util.find_files('/N/u/priyadarshini/Projects/Noise_signals/v'))

# Noise Signals
Nv = []
for noise_signal in files_v[0:1200]:
    snv, sr = librosa.load(noise_signal, sr=None)
    #Snv = librosa.stft(snv, n_fft=1024, hop_length=512)
    #Sn_noise = np.transpose(np.abs(librosa.stft(snv, n_fft=1024, hop_length=512)))
    Nv.append(np.transpose(np.abs(librosa.stft(snv, n_fft=1024, hop_length=512))))

# Clean Speech Signals    
Sv = []
for clean_signal in files_v[1200:2400]:
    ssv, sr = librosa.load(clean_signal, sr=None)
    #Ssv = librosa.stft(ssv, n_fft=1024, hop_length=512)
    #Sv_clean = np.transpose(np.abs(librosa.stft(ssv, n_fft=1024, hop_length=512)))
    Sv.append(np.transpose(np.abs(librosa.stft(ssv, n_fft=1024, hop_length=512))))    


# Mixed Noisy Speech Signals    
Xv = []
for mix_signal in files_v[2400:3600]:
    sxv, sr = librosa.load(mix_signal, sr=None)
    #Sxv = librosa.stft(sxv, n_fft=1024, hop_length=512)
    #Sv_mix = np.transpose(np.abs(librosa.stft(sxv, n_fft=1024, hop_length=512)))
    Xv.append(np.transpose(np.abs(librosa.stft(sxv, n_fft=1024, hop_length=512))))   

### Creating IBM matrix for validation spectograms

In [8]:
IBMv = Xv
for k in range(0,len(IBMv)) :
    for i in range(0,len(IBMv[k])):
        for j in range(0,len(IBMv[k][i])):
            if Sv[k][i][j] > Nv[k][i][j]:
                IBMv[k][i][j] = 1
            else:
                IBMv[k][i][j] = 0

In [13]:
max1 = 0
for spectogram in Xv :
    rows = len(spectogram)
    if rows >max1:
        max1 = rows
print(max1)

178


### Padding Validation matrices with zeroes

In [76]:
X_mv = [np.zeros((178,513))]*1200
for k in range(0,len(Xv)) :
    for i in range(0,len(Xv[k])):
        for j in range(0,len(Xv[k][i])):
            X_mv[k][i][j] = Xv[k][i][j]
            
IBM_mv = [np.zeros((178,513))]*1200
for k in range(0,len(IBMv)) :
    for i in range(0,len(IBMv[k])):
        for j in range(0,len(IBMv[k][i])):
            IBM_mv[k][i][j] = IBMv[k][i][j]   
            
            
           

In [77]:
X_new_v = np.array(X_mv).reshape(-1,10,513)  
IBM_new_v = np.array(IBM_mv).reshape(-1,10,513)

print(X_new_v.shape)
print(IBM_new_v.shape) 

(21360, 10, 513)
(21360, 10, 513)


### 1.8 Acheived MSE Loss of 0.055  on Validation signals

In [86]:
mse_v = 0
i = 0
for batch in batching_1(X_new_v, IBM_new_v, num_time_steps):
        i = i + 1
        X_batch_v, y_batch_v = batch
        mse_v = mse_v + loss.eval(feed_dict={x: X_batch_v, y: y_batch_v})
        
mse_v = mse_v/i        
print(mse_v)

0.055380636205275856


### 2.9 Test Signals - Creating spectograms for files in testing folder

In [80]:
files_t = np.asarray(librosa.util.find_files('/N/u/priyadarshini/Projects/Noise_signals/te'))

# Mixed Noisy Speech Signals    
Xt = []
St = []

for mix_signal in files_t[0:400]:
    sxt, sr = librosa.load(mix_signal, sr=None)
    Sxt = librosa.stft(sxt, n_fft=1024, hop_length=512)
    St.append(Sxt)
    Xt.append(np.transpose(np.abs(Sxt)))   

### 2.9 Producing Clean Test signals and Writing to the folder

In [84]:
for i in range(len(files_t)):
    s, sr = librosa.load(files_t[i], sr=None)
    test_c = librosa.stft(s, n_fft=1024, hop_length=512)
    test_abs = np.transpose(np.abs(test_c))
    test_abs = test_abs.reshape(1, -1, 513)
    test_denoise_trans = sess.run(outputs, feed_dict={x: test_abs})
    test_denoise_reshaped = test_denoise_trans.reshape(-1, 513) 
    test_denoise = np.transpose(test_denoise_reshaped)
    test_phase = np.divide(test_c, np.abs(test_c))
    test_denoise_phase = np.multiply(test_denoise, test_phase)
    sp = librosa.istft(test_denoise_phase,hop_length=512)
    librosa.output.write_wav('/N/u/priyadarshini/Projects/Test Clean Signals/tex_s_recons_' + str(i) + '.wav', sp, sr)

### Zipping the Denoised test signals 

In [2]:
import os
import zipfile

zf = zipfile.ZipFile("myzipfile.zip", "w")
for dirname, subdirs, files in os.walk("/N/u/priyadarshini/Projects/Test Clean Signals"):
    zf.write(dirname)
    for filename in files:
        zf.write(os.path.join(dirname, filename))
zf.close()